In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
import re
from datetime import datetime
## %pip install pycountry
import pycountry

In [2]:
url = "C:/Users/ferna/Desktop/IRONHACK/w2-project_pandas-ANDER-FERNANDEZ/Data/attacks.csv"
sharks_origin = pd.read_csv(url, encoding = "latin1")

In [3]:
## crear sharks, desde una copia de sharks_origin

sharks = sharks_origin.copy()

In [4]:
## Empiezo limpiando todas las columnas que serán necesarias para el análisis a realizar.
## Objetivo: Cuando el usuario introduzca Sexo, Pais y Actividad, le devuelva el porcentaje de ataques que se han realizado bajo esas condicioens


## Las palabras, "Not details", "Unknown", "Unconfirmed", "Invalid", "No shark", 0 y "." las cambio por NaN para ver mas rapidamente la falta de datos

for columna in sharks.columns:
    sharks[columna] = sharks[columna].replace(r'^\s*$', np.nan, regex=True)
    
for columna in sharks.columns:
    sharks[columna] = sharks[columna].replace("Unknown", np.nan)

for columna in sharks.columns:
    sharks[columna] = sharks[columna].replace("Unconfirmed", np.nan)
    
for columna in sharks.columns:
    sharks[columna] = sharks[columna].replace("No details", np.nan)

for columna in sharks.columns:
    sharks[columna] = sharks[columna].replace("Invalid", np.nan)
    
for columna in sharks.columns:
    sharks[columna] = sharks[columna].replace("0", np.nan)
    
for columna in sharks.columns:
    sharks[columna] = sharks[columna].replace(0, np.nan)

for columna in sharks.columns:
    sharks[columna] = sharks[columna].replace(0., np.nan)
    
for columna in sharks.columns:
    sharks[columna] = sharks[columna].replace("xx", np.nan)


In [5]:
## añado funciones para checkeo de nulos

def check_nulos(df: pd.DataFrame)-> None:  ## Buscar % de nulos
    
    nan_cols = df.isna().mean() * 100  # % de valores nulos
    
    nan_cols = nan_cols[nan_cols>0]
    
    display(f'N nan cols: {len(nan_cols)}')
    display(nan_cols)
    
def check_suma_nulos(df: pd.DataFrame)-> None:   ## Suma de valores nulos por columna
    
    nan_cols = df.isna().sum()
    
    nan_cols = nan_cols[nan_cols>0]
    
    display(nan_cols)

In [6]:
## En la columna Species observado que despues de un numero se indica numero de tiburones y no especie, se convierte a nan esos casos

def process_species(text):
    if isinstance(text, str) and text and text[0].isdigit():
        return np.nan  # Cambiar a NaN si comienza con un número
    else:
        return text
    
sharks['Species '] = sharks['Species '].apply(process_species)

In [7]:
## Para la columna Sex, cambio a NaN todo lo que no sean valores válidos

excepciones = ["F", "M", "N", "lli"]
def reemplazar_valor(df):
    if df in excepciones:
        return df
    else:
        return np.nan
    
sharks['Sex '] = sharks['Sex '].apply(reemplazar_valor)

sharks['Sex '].unique()

array(['F', 'M', nan, 'lli', 'N'], dtype=object)

In [8]:
## La columna país la voy a limpiar usando la bilbitoca pycountry. Todo lo que no sean valores válidos se convierten a NaN

def verificar_pais(valor):
    if isinstance(valor, str):
        valor = valor.lower()  # Convertir a minúsculas para no distinguir entre mayúsculas y minúsculas
        for country in pycountry.countries:
            if valor == country.name.lower() or valor == country.alpha_2.lower() or valor == country.alpha_3.lower():
                return valor
    return np.nan

sharks['Country'] = sharks['Country'].apply(verificar_pais)

sharks['Country'].unique()

array(['usa', 'australia', 'mexico', 'brazil', nan, 'south africa',
       'thailand', 'costa rica', 'maldives', 'bahamas', 'new caledonia',
       'ecuador', 'malaysia', 'libya', 'cuba', 'mauritius', 'new zealand',
       'spain', 'samoa', 'solomon islands', 'japan', 'egypt', 'comoros',
       'french polynesia', 'united kingdom', 'united arab emirates',
       'philippines', 'indonesia', 'china', 'fiji', 'dominican republic',
       'cayman islands', 'aruba', 'mozambique', 'puerto rico', 'italy',
       'greece', 'france', 'papua new guinea', 'kiribati', 'israel',
       'jamaica', 'guam', 'seychelles', 'belize', 'nigeria', 'tonga',
       'canada', 'croatia', 'saudi arabia', 'chile', 'kenya', 'malta',
       'madagascar', 'panama', 'somalia', 'norway', 'senegal', 'yemen',
       'sierra leone', 'liberia', 'vanuatu', 'honduras', 'sri lanka',
       'uruguay', 'india', 'marshall islands', 'hong kong', 'el salvador',
       'angola', 'bermuda', 'montenegro', 'tunisia', 'namibia',
     

In [9]:
## Elimino todas las filas en las que todas las columnas sean NaN

sharks = sharks.dropna(how="all")

In [10]:
## Voy a unificar todas las actividades en los grupos más generales

sharks.loc[sharks['Activity'].str.contains('surf', case=False, na=False), 'Activity'] = "Surfing"
sharks.loc[sharks['Activity'].str.contains('Crossing', case=False, na=False), 'Activity'] = "Swimming"
sharks.loc[sharks['Activity'].str.contains('boogie', case=False, na=False), 'Activity'] = "Body boarding"
sharks.loc[sharks['Activity'].str.contains('Swim', case=False, na=False), 'Activity'] = "Swimming"
sharks.loc[sharks['Activity'].str.contains('fishing', case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('boat', case=False, na=False), 'Activity'] = "Boating"
sharks.loc[sharks['Activity'].str.contains('overboard', case=False, na=False), 'Activity'] = "Boating"
sharks.loc[sharks['Activity'].str.contains('canoeing', case=False, na=False), 'Activity'] = "Kayaking"
sharks.loc[sharks['Activity'].str.contains('Crawling', case=False, na=False), 'Activity'] = "Walking"
sharks.loc[sharks['Activity'].str.contains('Canoe', case=False, na=False), 'Activity'] = "Kayaking"
sharks.loc[sharks['Activity'].str.contains('floating', case=False, na=False), 'Activity'] = "Swimming"
sharks.loc[sharks['Activity'].str.contains('sailing', case=False, na=False), 'Activity'] = "Boating"
sharks.loc[sharks['Activity'].str.contains('board', case=False, na=False), 'Activity'] = "Surfing"
sharks.loc[sharks['Activity'].str.contains('playing', case=False, na=False), 'Activity'] = "Standing"
sharks.loc[sharks['Activity'].str.contains('fishing',  case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('clamming',  case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('seine net',  case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('kakaying',  case=False, na=False), 'Activity'] = "Kayaking"
sharks.loc[sharks['Activity'].str.contains('crabbing',  case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('splashing', case=False, na=False), 'Activity'] = "Standing"
sharks.loc[sharks['Activity'].str.contains('stamding', case=False, na=False), 'Activity'] = "Standing"
sharks.loc[sharks['Activity'].str.contains('Sitting', case=False, na=False), 'Activity'] = "Standing"
sharks.loc[sharks['Activity'].str.contains('paddl', case=False, na=False), 'Activity'] = "Surfing"
sharks.loc[sharks['Activity'].str.contains('shoot', case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('jumping', case=False, na=False), 'Activity'] = "Swimming"
sharks.loc[sharks['Activity'].str.contains('shell',  case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('catch', case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('shark', case=False, na=False), 'Activity'] = "Shark related"
sharks.loc[sharks['Activity'].str.contains('lying', case=False, na=False), 'Activity'] = "Standing"
sharks.loc[sharks['Activity'].str.contains('lobster',  case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('Sink',  case=False, na=False), 'Activity'] = "Diving"
sharks.loc[sharks['Activity'].str.contains('collect',  case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('underwater',  case=False, na=False), 'Activity'] = "Diving"
sharks.loc[sharks['Activity'].str.contains('harpoon',  case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('submarine', case=False, na=False), 'Activity'] = "Boating"
sharks.loc[sharks['Activity'].str.contains('ridi', case=False, na=False), 'Activity'] = "Boating"
sharks.loc[sharks['Activity'].str.contains('washing', case=False, na=False), 'Activity'] = "Washing"
sharks.loc[sharks['Activity'].str.contains('walking', case=False, na=False), 'Activity'] = "Walking"
sharks.loc[sharks['Activity'].str.contains('Hurricane', case=False, na=False), 'Activity'] = "Sea Disaster"
sharks.loc[sharks['Activity'].str.contains('tsunami', case=False, na=False), 'Activity'] = "Sea Disaster"
sharks.loc[sharks['Activity'].str.contains('Clam',  case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('oyster',  case=False, na=False), 'Activity'] = "Fishing"
sharks.loc[sharks['Activity'].str.contains('vessel', case=False, na=False), 'Activity'] = "Boating"
sharks.loc[sharks['Activity'].str.contains('cliff', case=False, na=False), 'Activity'] = "Diving"
sharks.loc[sharks['Activity'].str.contains('Unknown,', case=False, na=False), 'Activity'] = np.nan
sharks.loc[sharks['Activity'].str.contains('Boeing', case=False, na=False), 'Activity'] = "Air Disaster"
sharks.loc[sharks['Activity'].str.contains('research', case=False, na=False), 'Activity'] = "Conducting research"
sharks.loc[sharks['Activity'].str.contains('murder', case=False, na=False), 'Activity'] = "Murder"

In [11]:
## En la columna Fatal, dejar solo "N" y "Y", quitar espacios cuando es valor valido con espacio

sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].str.upper()
valores_validos = ['Y', 'N']

sharks['Fatal (Y/N)'] = sharks['Fatal (Y/N)'].apply(lambda x: x if x in valores_validos else np.nan)

sharks['Fatal (Y/N)'].unique()

array(['N', 'Y', nan], dtype=object)

In [12]:
## cambiar los valores Boat y Boatomg por Boating

sharks['Type'] = sharks['Type'].replace(['Boat', 'Boatomg'], 'Boating')
sharks.Type.unique()

array(['Boating', 'Unprovoked', nan, 'Provoked', 'Questionable',
       'Sea Disaster'], dtype=object)

In [13]:
## La columna Type la unifico en Unprovoked, Provoked y Questionable

sharks.loc[sharks['Type'].str.contains('Sea Disaster', case=False, na=False), 'Type'] = 'Unprovoked'
sharks.loc[sharks['Type'].str.contains('Boating', case=False, na=False), 'Type'] = 'Unprovoked'


sharks.Type.unique()

array(['Unprovoked', nan, 'Provoked', 'Questionable'], dtype=object)

In [14]:
sharks["Species "].unique()

array(['White shark', nan, 'Tiger shark, 3m', ...,
       '"a black-tipped shark"', 'Blue pointers',
       'Said to involve a grey nurse shark that leapt out of the water and  seized the boy but species identification is questionable'],
      dtype=object)

In [15]:
sharks.shape

(6309, 24)

In [82]:
## observado que despues de un numero se indica numero de tiburones y no especie, se convierte a nan esos casos

def process_species(text):
    if isinstance(text, str) and text and text[0].isdigit():
        return np.nan  # Cambiar a NaN si comienza con un número
    else:
        return text
    
sharks['Species '] = sharks['Species '].apply(process_species)

## muchos datos seguidos de una coma, todo lo que venga despues de comas, lo borro

sharks['Species '] = sharks['Species '].str.split(',').str[0]

## La columna species, no tiene relevancia para el analisis, pero procedo a limpiarla

## Despues de las comas solo viene información innecesaria, separo por la coma y dejo solo la parte de antes
def remove_after_dot_or_x(text):
    if isinstance(text, str):
        parts = re.split('[\.x]', text)
        if len(parts) > 1:
            return parts[0].strip()
        elif re.match(r'^\d+$', text):
            return np.nan
    return text
    
sharks['Species '] = sharks['Species '].apply(remove_after_dot_or_x)


## observado que despues de un numero se indica numero de tiburones y no especie, se convierte a nan esos casos

def process_species(text):
    if isinstance(text, str) and text and text[0].isdigit():
        return np.nan  # Cambiar a NaN si comienza con un número
    else:
        return text
    
sharks['Species '] = sharks['Species '].apply(process_species)

## en la columna species, hay "No shark..","Questionable shark involve" los voy a cambiar por nan

sharks.loc[sharks['Species '].str.contains('No shark', case=False, na=False), 'Species '] = np.nan

sharks.loc[sharks['Species '].str.contains('Questionable', case=False, na=False), 'Species '] = np.nan

sharks.loc[sharks['Species '].str.contains('Invalid', case=False, na=False), 'Species '] = np.nan

sharks.loc[sharks['Species '].str.contains('Not confirmed', case=False, na=False), 'Species '] = np.nan

## en la columna species, si contiene nombre de especie, que me devuelva la especie bien

sharks.loc[sharks['Species '].str.contains('White', case=False, na=False), 'Species '] = "White Shark"
sharks.loc[sharks['Species '].str.contains('Hammer', case=False, na=False), 'Species '] = "Hammerhead Shark"
sharks.loc[sharks['Species '].str.contains('Tiger', case=False, na=False), 'Species '] = "Tiger Shark"
sharks.loc[sharks['Species '].str.contains('Bull', case=False, na=False), 'Species '] = "Bull Shark"
sharks.loc[sharks['Species '].str.contains('Blue', case=False, na=False), 'Species '] = "Blue Shark"
sharks.loc[sharks['Species '].str.contains('Mako', case=False, na=False), 'Species '] = "Mako Shark"
sharks.loc[sharks['Species '].str.contains('Threshe', case=False, na=False), 'Species '] = "Thresher Shark"
sharks.loc[sharks['Species '].str.contains('Basking', case=False, na=False), 'Species '] = "Basking Shark"
sharks.loc[sharks['Species '].str.contains('Oceanic', case=False, na=False), 'Species '] = "Oceanic Shark"
sharks.loc[sharks['Species '].str.contains('Pelagic', case=False, na=False), 'Species '] = "Pelagic Thresher Shark"
sharks.loc[sharks['Species '].str.contains('Lemon', case=False, na=False), 'Species '] = "Lemon Shark"
sharks.loc[sharks['Species '].str.contains('Nurse', case=False, na=False), 'Species '] = "Nurse Shark"
sharks.loc[sharks['Species '].str.contains('Goblin', case=False, na=False), 'Species '] = "Goblin Shark"
sharks.loc[sharks['Species '].str.contains('Blacktip', case=False, na=False), 'Species '] = "Blacktip Shark"
sharks.loc[sharks['Species '].str.contains('Galapagos', case=False, na=False), 'Species '] = "Galapagos Shark"
sharks.loc[sharks['Species '].str.contains('Caribbean', case=False, na=False), 'Species '] = "Caribbean Reef Shark"
sharks.loc[sharks['Species '].str.contains('Dusky', case=False, na=False), 'Species '] = "Dusky Shark"
sharks.loc[sharks['Species '].str.contains('Whale', case=False, na=False), 'Species '] = "Whale Shark"
sharks.loc[sharks['Species '].str.contains('Largetooth', case=False, na=False), 'Species '] = "Largetooth Sawfish"
sharks.loc[sharks['Species '].str.contains('Silky', case=False, na=False), 'Species '] = "Silky Shark"
sharks.loc[sharks['Species '].str.contains('Spinner', case=False, na=False), 'Species '] = "Spinner Shark"
sharks.loc[sharks['Species '].str.contains('Porbeagle', case=False, na=False), 'Species '] = "Porbeagle Shark"
sharks.loc[sharks['Species '].str.contains('Reef', case=False, na=False), 'Species '] = "Reef Shark"
sharks.loc[sharks['Species '].str.contains('Dogfish', case=False, na=False), 'Species '] = "Dogfish Shark"
sharks.loc[sharks['Species '].str.contains('Angel', case=False, na=False), 'Species '] = "Angel Shark"
sharks.loc[sharks['Species '].str.contains('Zebra', case=False, na=False), 'Species '] = "Zebra Shark"
sharks.loc[sharks['Species '].str.contains('Cow', case=False, na=False), 'Species '] = "Cow Shark"
sharks.loc[sharks['Species '].str.contains('Sand', case=False, na=False), 'Species '] = "Sand Shark"
sharks.loc[sharks['Species '].str.contains('Horn', case=False, na=False), 'Species '] = "Horn Shark"
sharks.loc[sharks['Species '].str.contains('Gulper', case=False, na=False), 'Species '] = "Gulper Shark"
sharks.loc[sharks['Species '].str.contains('Broadnose Sevengill', case=False, na=False), 'Species '] = "Broadnose Sevengill Shark"
sharks.loc[sharks['Species '].str.contains('eel', case=False, na=False), 'Species '] = np.nan
sharks.loc[sharks['Species '].str.contains('Wobbegong', case=False, na=False), 'Species '] = "Wobbegong Shark"
sharks.loc[sharks['Species '].str.contains('very large', case=False, na=False), 'Species '] = np.nan
sharks.loc[sharks['Species '].str.contains('brown', case=False, na=False), 'Species '] = "Brown Shark"


In [83]:
## quito las filas en las que todos los valores son nulos

sharks = sharks.dropna(how="all")

In [84]:
## Hay 25 columnas, compruebo que no haya filas con mas de 23 columnas con NAN.

mas22nan = sharks.isna().sum(axis=1) > 22
sharks2 = sharks[mas22nan]
sharks2

,index,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23


In [85]:
## los que tienen más de 11, no me proporcionan informacion suficiente para mi analisis. Borro

mas11nan = sharks.isna().sum(axis=1) > 11
sharks2 = sharks[mas11nan]
sharks2

,index,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23


In [73]:
## elimino las columnas

## sharks = sharks.dropna(thresh=sharks.shape[1] - 11)

## sharks

,index,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,0,2018.06.25,25-Jun-2018,2018.0,Unprovoked,usa,California,"Oceanside, San Diego County",Surfing,Julie Wolfe,F,57,NaN,N,18h00,White Shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2,2018.06.09,09-Jun-2018,2018.0,NaN,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,australia,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,NaN,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,4,2018.06.04,04-Jun-2018,2018.0,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,Tiger Shark,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6289,6296,ND.0006,Before 1906,NaN,Unprovoked,australia,New South Wales,NaN,Swimming,Arab boy,M,NaN,FATAL,Y,NaN,NaN,"L. Becke in New York Sun, 9/9/1906; L. Schultz...",ND-0006-ArabBoy-Prymount.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0006,ND.0006,7.0,NaN,NaN
6290,6297,ND.0005,Before 1903,NaN,Unprovoked,australia,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN,NaN
6291,6298,ND.0004,Before 1903,NaN,Unprovoked,australia,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN,NaN
6292,6299,ND.0003,1900-1905,NaN,Unprovoked,usa,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN,NaN


In [54]:
## Elimino las que son NaN en todos los campos de analisis y las que tienen dos campos nulos
sharks = sharks.dropna(subset=['Activity', 'Country', 'Sex '], how='all')

mask = sharks[['Activity', 'Country', 'Sex ']].isna().sum(axis=1) >= 2

sharks.drop(sharks[mask].index, inplace=True)

In [75]:
sharks.shape

(5772, 25)

In [70]:
actividad_pais = sharks.groupby(['Country', 'Activity', 'Sex ']).size().reset_index(name='Count')
total_registros = len(sharks)
actividad_pais['Percentage'] = (actividad_pais['Count'] / total_registros) * 100
actividad_pais = actividad_pais.sort_values(by='Percentage', ascending=False).T
actividad_pais

,830,834,753,125,64,127,656,658,634,833,829,841,551,14,819,155,840,198,731,34,200,22,496,808,90,818,742,126,728,801,512,379,462,659,108,476,783,124,484,563,282,102,654,85,815,816,138,340,581,300,117,510,647,754,114,113,680,247,389,452,752,846,491,234,342,471,237,460,46,486,164,620,446,542,657,684,367,662,573,807,266,230,227,135,436,679,363,319,845,757,509,231,615,393,594,635,229,364,116,630,271,644,645,704,400,153,292,207,675,165,193,275,184,390,172,63,66,61,534,782,267,306,150,461,154,655,498,464,166,837,698,592,730,381,727,258,408,405,387,532,708,391,701,13,225,453,440,352,506,318,640,804,39,376,377,212,107,421,568,324,488,262,627,571,404,800,590,105,858,769,74,137,851,371,653,437,21,555,750,395,217,813,505,134,494,2,485,697,347,587,482,548,614,678,53,859,695,687,339,264,384,511,314,368,599,602,296,295,291,290,287,284,661,277,490,41,268,323,670,366,606,673,351,257,525,281,772,699,557,799,423,565,564,171,562,700,744,540,163,560,159,158,432,463,188,95,435,144,143,439,545,101,442,131,764,454,778,123,547,187,346,407,76,219,707,406,710,475,706,705,579,604,554,514,529,572,603,586,605,515,601,543,516,544,585,517,530,583,549,550,546,531,582,613,612,584,552,611,610,553,609,608,513,607,528,600,518,574,523,596,538,595,537,576,575,566,541,524,593,567,569,536,535,570,577,578,522,597,591,556,519,598,527,588,580,533,558,559,589,520,521,526,539,561,0,616,773,775,776,777,779,780,781,784,785,786,787,788,789,790,791,792,793,794,774,771,796,770,746,747,748,749,751,755,756,758,759,760,761,762,763,765,766,767,768,795,797,617,831,835,836,838,839,842,843,844,847,848,849,850,852,853,854,855,856,857,832,828,798,827,802,803,805,806,809,810,811,812,814,817,820,821,822,823,824,825,826,745,743,741,643,648,649,650,651,652,660,663,664,665,666,667,668,669,671,672,674,677,646,642,740,641,618,619,621,622,623,624,625,626,628,629,631,632,633,636,637,638,639,681,682,683,685,719,720,721,722,723,724,725,726,729,732,733,734,735,736,737,738,739,718,717,716,694,686,688,689,690,691,692,693,696,715,702,703,709,711,712,713,714,676,430,508,169,152,156,157,160,161,162,167,168,170,149,173,174,175,176,177,178,179,180,151,148,182,130,115,118,119,120,121,122,128,129,132,147,133,136,139,140,141,142,145,146,181,183,243,226,215,216,218,220,221,222,223,224,228,213,232,233,235,236,238,239,240,241,214,211,185,197,186,189,190,191,192,194,195,196,199,210,201,202,203,204,205,206,208,209,112,111,110,38,29,30,31,32,33,35,36,37,40,27,42,43,44,45,47,48,49,50,28,26,109,11,3,4,5,6,7,8,9,10,12,25,15,16,17,18,19,20,23,24,51,52,54,93,83,84,86,87,88,89,91,92,94,55,96,97,98,99,100,103,104,106,82,81,80,79,56,57,58,59,60,62,65,67,68,69,70,71,72,73,75,77,78,242,244,507,420,412,413,414,415,416,417,418,419,422,410,424,425,426,427,428,429,1,431,411,409,434,386,373,374,375,378,380,382,383,385,388,403,392,394,396,397,398,399,401,402,433,438,245,492,477,478,479,480,481,483,487,489,493,473,495,497,499,500,501,502,503,504,474,472,441,455,443,444,445,447,448,449,450,451,456,470,457,458,459,465,466,467,468,469,372,370,369,294,279,280,283,285,286,288,289,293,297,276,298,299,301,302,303,304,305,307,278,274,365,255,246,248,249,250,251,252,253,254,256,273,259,260,261,263,265,269,270,272,308,309,310,353,338,341,343,344,345,348,349,350,354,311,355,356,357,358,359,360,361,362,337,336,335,334,312,313,315,316,317,320,321,322,325,326,327,328,329,330,331,332,333,860
Country,usa,usa,usa,australia,australia,australia,south africa,south africa,south africa,usa,usa,usa,papua new guinea,australia,usa,bahamas,usa,brazil,usa,australia,brazil,australia,new zealand,usa,australia,usa,usa,australia,usa,usa,new zealand,italy,mexico,south africa,australia,mozambique,usa,australia,new caledonia,papua new guinea,fiji,australia,south africa,australia,usa,usa,australia,hong kong,philippines,french polynesia,australia,new zealand,south africa,usa,australia,australia,spain,cuba,italy,mexico,usa,usa,new zealand,cuba,india,mozambique,cuba,mexico,australia,new caledonia,bahamas,solomon islands,mexico,panama,south africa,sri lanka,iraq,south

In [76]:
## Creo un DF a parte para hacer mi análisis.

sharks_analisis_final = sharks[['Activity', 'Country', 'Sex ']]

sharks_analisis_final

,Activity,Country,Sex
0,Surfing,usa,F
1,Standing,usa,F
2,Surfing,usa,M
3,Surfing,australia,M
4,Free diving,mexico,M
...,...,...,...
6289,Swimming,australia,M
6290,Diving,australia,M
6291,Pearl diving,australia,M
6292,Swimming,usa,M


In [66]:
sexo_usuario = input("Ingresa tu sexo (M/F/N/illi): ").strip().lower()
pais_usuario = input("Ingresa tu país: ").strip().lower()
actividad_usuario = input("Ingresa tu actividad favorita: ").strip().lower()

sharks_analisis_final['Sex '] = sharks_analisis_final['Sex '].str.lower()
sharks_analisis_final['Country'] = sharks_analisis_final['Country'].str.lower()
sharks_analisis_final['Activity'] = sharks_analisis_final['Activity'].str.lower()

datos_usuario = sharks_analisis_final[(sharks_analisis_final['Sex '] == sexo_usuario) & (sharks_analisis_final['Country'] == pais_usuario) & (sharks_analisis_final['Activity'] == actividad_usuario)]

total_registros = len(sharks_analisis_final)
porcentaje_posibilidad = (len(datos_usuario) / total_registros) * 100

print(f"El: {porcentaje_posibilidad:.2f}% de los ataques registrados han sido bajo esos parametros")

Ingresa tu sexo (M/F/N/illi): m
Ingresa tu país: usa
Ingresa tu actividad favorita: surfing
El: 12.11% de los ataques registrados han sido bajo esos parametros


C:\Users\ferna\AppData\Local\Temp\ipykernel_14492\3732196618.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks_analisis_final['Sex '] = sharks_analisis_final['Sex '].str.lower()
C:\Users\ferna\AppData\Local\Temp\ipykernel_14492\3732196618.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks_analisis_final['Country'] = sharks_analisis_final['Country'].str.lower()
C:\Users\ferna\AppData\Local\Temp\ipykernel_14492\3732196618.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [27]:
sharks["Age"].unique()

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8',
       '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
       '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
       '8 or 10', '84', '30 or 36', '6½', '21 & ?', '75', '33 or 37',
       'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28', '20?',
       "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly', 'mid-20s',
       'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50', '18 to 22',
       'adult', '9 & 12', '? & 19', '9 months', '25 to 35', '23 & 26',
     

In [77]:
non_numeric_count = sharks['Age'][pd.to_numeric(sharks['Age'], errors='coerce').isna()].value_counts().sum()
non_numeric_count

94

In [78]:
## en la columna age, hay valores que vienen como "23 & 35", indica que hubo dos victimas en ese ataque, creo una linea nueva por cada victima para poder contar el numero de victimas ademas del numero de ataques

def transformar_edad(valor):
    if pd.notna(valor):
        edades = []
        if '&' in valor:
            edades.extend([int(edad.strip()) for edad in valor.split('&') if edad.strip().isdigit()])
        if 'or' in valor:
            edades.extend([int(edad.strip()) for edad in valor.split('or') if edad.strip().isdigit()])
        if 'to' in valor:
            edades.extend([int(edad.strip()) for edad in valor.split('to') if edad.strip().isdigit()])
        if ',' in valor:
            valores = valor.split(',')
            for val in valores:
                edades.extend([int(edad.strip()) for edad in val.split('&') if edad.strip().isdigit()])
        if edades:
            min_edad = min(edades)
            max_edad = max(edades)
            nuevas_filas = [{'Age': int(min_edad)}, {'Age': int(max_edad)}]
            return nuevas_filas
        elif valor.isdigit():
            return [{'Age': int(valor)}]
        else:
            return [{'Age': np.nan}]
    else:
        return [{'Age': np.nan}]
   
nuevas_filas = []
    
for valor in sharks['Age']:
    nuevas_filas.extend(transformar_edad(valor))
    
nuevo_sharks = pd.DataFrame(nuevas_filas)

nuevo_sharks['Age'] = pd.to_numeric(nuevo_sharks['Age'], errors='coerce')

## en los casos de: "24 & 36" se ha creado una nueva linea, para los casos en los que hay 3 numeros, no soy capaz
## los casos en los que hay mas de dos numeros son muy pocos y se han despreciado. 
##El numero de lineas en el DF nuevo_sharks indica el numero de victimas total que es de 5882 en 5772 ataques

nuevo_sharks

,Age
0,57.0
1,11.0
2,48.0
3,NaN
4,NaN
...,...
5805,NaN
5806,NaN
5807,NaN
5808,NaN
